In [1]:
from pyspark.sql.functions import *
import pandas as pd
import numpy as np
from delta_utils.write import WriteDeltaHive


universe = spark.table('dev_derived_ouniverse.output_total_universe_nl_enriched')
sr_areas = spark.table('data_user_celeste.postcodegebied_per_srm')

In [2]:
universe = universe.drop('4digitPC').drop('postalCode_cleaned')

In [3]:
sr_areas = sr_areas.withColumnRenamed('Postcode', '4digitPC').withColumnRenamed('SRM', 'SALESREP').drop('PC2')

In [4]:
universe = universe.drop('SALESREP')
universe = universe.withColumn('4digitPC', substring('postalCode', 0, 4))

universe = universe.join(sr_areas, on='4digitPC', how='left')

In [5]:
universe.select('SALESREP').show(100, False)

+--------------------+
SALESREP |
+--------------------+
Ron Quartel |
Kelly Driessen |
Kelly Driessen |
Marc Bontekoe |
Michael van Mierlo |
Ron Quartel |
Bas Mantel |
Marc Bontekoe |
Kelly Driessen |
Frank de Graaf |
Michael van Mierlo |
Danny van de Put |
Bas Mantel |
Marc Bontekoe |
Kelly Driessen |
Koen Hartmann |
null |
Kelly Driessen |
Bas Mantel |
Peter Renkema |
Ron Bonewald |
Niels Behrens |
Geartsje Stegenga |
Kelly Driessen |
Marc Bontekoe |
Niels Behrens |
Geartsje Stegenga |
Kelly Driessen |
Geartsje Stegenga |
Ron Quartel |
Ron Bonewald |
Frank de Graaf |
Kelly Driessen |
Ron Bonewald |
Geartsje Stegenga |
Koen Hartmann |
Johan van Wageningen|
Leo Vink |
Bas Mantel |
Koen Hartmann |
Leo Vink |
Shandra de Graaff |
Johan van Wageningen|
Ron Quartel |
Niels Behrens |
Geartsje Stegenga |
Bert Timmerman |
Geartsje Stegenga |
Niels Behrens |
Koen Hartmann |
Bert Timmerman |
Geartsje Stegenga |
Bas Mantel |
Frank de Graaf |
Bas Mantel |
Marc Bontekoe |
Bert Timmerman |
Geartsje Stegenga |
Geartsje Stegenga |
Peter Renkema |
Bas Mantel |
Johan van Wageningen|
Johan van Wageningen|
Shandra de Graaff |
Geartsje Stegenga |
Geartsje Stegenga |
Geartsje Stegenga |
Ron Bonewald |
Marc Bontekoe |
Leo Vink |
Ron Bonewald |
Frank de Graaf |
Ron Quartel |
Shandra de Graaff |
Koen Hartmann |
Frank de Graaf |
Ron Quartel |
Michael van Mierlo |
Danny van de Put |
Ron Quartel |
Michael van Mierlo |
Koen Hartmann |
null |
Michael van Mierlo |
Johan van Wageningen|
Bas Mantel |
Frank de Graaf |
Bert Timmerman |
Leo Vink |
Niels Behrens |
Ron Quartel |
Danny van de Put |
Rowanne Pos |
Johan van Wageningen|
Marc Bontekoe |
Shandra de Graaff |
Shandra de Graaff |
Frank de Graaf |
Rowanne Pos |
Marc Bontekoe |
+--------------------+
only showing top 100 rows

In [6]:
deltaTable = '/mnt/datamodel/dev/derived/ouniverse/output/total_universe_nl_enriched'
hiveTable = "dev_derived_ouniverse.output_total_universe_nl_enriched"
WriteDeltaHive(spark, deltaTable, hiveTable, universe).main()

In [7]:
top75 = spark.table('data_user_celeste.top75_per_sr')

In [8]:
top75.count()

Out[3]: 1171

In [9]:
top75 = top75.withColumnRenamed('SAP ID', 'SAPID')
top75 = top75.withColumn('SAPID', regexp_replace('SAPID', "-", ""))
top75 = top75.withColumn('top75', lit(1))

In [10]:
top75_for_join = top75.select('SAPID', 'top75')
inner_un_top75 = universe.join(top75, on='SAPID', how='inner')
top75_nm = top75.join(inner_un_top75, on='SAPID', how='leftanti')

In [11]:
# matches = fuzzyMatch(file1=universe, file2=top75_nm, name_source1='universe', name_source2='top75', name_colname1='name', name_colname2='Account Name', addres_colname1='address', addres_colname2='Billing Street', id_colname1='operatorId', id_colname2='Account ID', zipcode_colname1='postalCode', zipcode_colname2='Billing Zip/Postal Code').fuzzy_match()

In [12]:
universe_sap_matches = universe.join(top75_for_join, on='SAPID', how='left')

In [13]:
fz_matches_top75 = spark.table('data_user_celeste.top75_match_universe')
fz_matches_top75 = fz_matches_top75.filter(col('validation') == 1)
top75_sap_id_join = top75.select('SAPID', 'Account ID').withColumnRenamed('SAPID', 'SAPID_match')
fz_matches_top75 = fz_matches_top75.join(top75_sap_id_join, on='Account ID', how='left')
fz_matches_top75_bool = fz_matches_top75.select('operatorId', 'SAPID_match').withColumn('top75_match', lit(1))

In [14]:
universe_fuzzy_sap_matches = universe_sap_matches.join(fz_matches_top75_bool, on='operatorId', how='left')
universe_fuzzy_sap_matches = universe_fuzzy_sap_matches.withColumn('top75', when(col('top75_match') == 1, 1).otherwise(col('top75')))
universe_fuzzy_sap_matches = universe_fuzzy_sap_matches.withColumn('top75', when(col('SAPID_match').isNotNull(), 1).otherwise(col('SAPID')))
universe_fuzzy_sap_matches = universe_fuzzy_sap_matches.drop('top75_match').drop('SAPID_match')

In [15]:
match_sap_ids = universe_fuzzy_sap_matches.select('SAPID').filter(col('SAPID').isNotNull())

In [16]:
remaining_top75 = top75.join(match_sap_ids, on='SAPID', how='leftanti')

In [17]:
display(remaining_top75)

SAPID,Account ID,Account Owner,Customer Group,Sales office -Key,Account Name,Billing Street,Billing Zip/Postal Code,Billing City,Parent Account,Phone,Website,Has Webshop Account,Last Visit Date,Last Webshop Order Date_,top75
100368092951002110,0010N00004nWIzK,Niels Behrens,TOP75,NBE,Restaurant Op de Haven,Helling 7,4101 AE,CULEMBORG,null,31345510324,null,null,null,",",1
000249170151002110,0010N00004nWOVq,Geartsje Stegenga,TOP75,GST,Restaurant De Kajuit,Jachthaven de Domp 1,8605 CP,SNEEK,null,31515432503,null,null,null,",",1
100367079351002110,0010N00004nWGQW,Bert Timmerman,TOP75,BTI,Restaurant De Landbouw,Grote Markt 38,4461 AJ,GOES,null,31653780389,null,null,null,",",1
100367187051002110,0010N00004nWCFn,Geartsje Stegenga,TOP75,GST,Restaurant Hotel Eurohotel,Europaplein 20,8915 CL,LEEUWARDEN,null,31582131113,null,null,null,",",1
100368967851002110,0010N00004nWgd7,Niels Behrens,TOP75,NBE,Brunotti Beachclub,Noordoever Oostvoornse Meer,3233 NC,OOSTVOORNE,null,31181263130,null,null,null,",",1
100367076051002110,0010N00004nWGQm,Bert Timmerman,TOP75,BTI,Restaurant Dickens & Jones,Grote Markt 40,4811 XS,BREDA,null,31765211080,null,null,null,",",1
100501731451002110,0010N00004nWmgk,Ron Quartel,TOP75,RQU,Haagse Poort,Prinses beatrixlaan 35,2595 AK,DEN HAAG,null,null,null,null,null,",",1
100367523151002110,0010N00004nW97e,Johan van Wageningen,TOP75,JVW,Van Roessel Boerderijprodukten,Dorpsstraat 63,5051 CJ,GOIRLE,null,31135341484,null,null,null,",",1
100368578651002110,0010N00004nWFjv,Koen Hartmann,TOP75,KHAR,Wereldtuinen Mondo Verde,Groene Wereld 10,6372 PW,LANDGRAAF,null,31653234756,null,null,null,",",1
100368682851002110,0010N00004nW1Bo,Bas Mantel,TOP75,BMA,Sindy's Catering,Broekweg 5,7255 KP,HENGELO GLD,null,31575464532,null,null,null,"8-7-2020,",1


In [18]:
remaining_top75.count()

Out[16]: 576

In [19]:
display(universe_fuzzy_sap_matches)

operatorId,SAPID,4digitPC,postalCode_cleaned,PlaceIDGoogle,osmId,ohubID,name,address,postalCode,city,Latitude,Longitude,businessType,Website,cuisineType,NameGoogle,AddressGoogle,PostalCodeGoogle,CityGoogle,LatitudeGoogle,LongitudeGoogle,businessTypeGoogle,WebsiteGoogle,type,uid,nameOSM,AddressOSM,PostalcodeOSM,cityOSM,LatitudeOSM,LongitudeOSM,cuisineTypeOSM,websiteOSM,concatID,nameOhub,postalcodeOhub,addressOhub,cityOhub,source,globalChannel,chain,buying,valueTier,valueTierDescription,d_buyer,id,keyword,uniqueProductCount,SAP_DEX_ID,is_visited,cp_mobileNumber,phone,countryGoogle,InternationalPhoneNumberGoogle,uniqueID,take_away,isTargetUniverse,menuHasTerras,primary_topcategory,businessTypeOSM,channelOhub,menuDishes,globalListChannels,operatorLifeCycle_Stage,SALESREP,top75
/ / 017fb319-a22c-3fc2-8854-f844aaaaa855,null,2360,2360,null,null,017fb319-a22c-3fc2-8854-f844aaaaa855,"Verhoeven, Johannes",null,2360,Oud-turnhout,null,null,null,null,Unknown,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NL~FUZZIT~B89CA6EB-C14B-498E-B300-74077B2B62B7,"Verhoeven, Johannes",2360,Steenweg op Ravels 223-1,Oud-turnhout,NoBusinessTypeSource,Other,Unknown,true,null,null,null,null,null,null,null,null,null,null,null,,null,false,0,null,null,null,null,null,[Other],E0B0-low,Ron Quartel,null
/ / 0346d836-4357-3e23-88e9-c612b5458d4f,null,7131,7131,null,null,0346d836-4357-3e23-88e9-c612b5458d4f,APOTHEEK LICHTENVOORDE BV,null,7131DJ,Lichtenvoorde,null,null,null,null,Unknown,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NL~FUZZIT~40C743E8-0838-41C0-A320-C825FD86CA0F,APOTHEEK LICHTENVOORDE BV,7131DJ,CAREL LOOIERSTRAAT 2-01,Lichtenvoorde,NoBusinessTypeSource,Other,Unknown,true,8,€36.0 - €156.0 (average value = 96.0),1,null,null,null,null,null,null,null,null,,null,false,0,null,null,null,null,null,[Other],E0B0-low,Kelly Driessen,null
/ / 038fe63f-d6d9-4860-b91f-090da7606f41,100372182851002110,7041,7041,null,null,038fe63f-d6d9-4860-b91f-090da7606f41,Restaurant De Pleisterplaats,De Immenhorst 17,7041KE,'s-heerenberg,51.8698249,6.2591411,Restaurants,null,Unknown,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NL~DEX~1003721828,Restaurant De Pleisterplaats,7041KE,De Immenhorst 17,'s-heerenberg,OHUB,Other Restaurant,Unknown,null,3,€304.0 - €319.0 (average value = 312.0),0,null,null,null,null,null,null,null,null,,null,false,1,null,null,null,Restaurants,null,"[Other, Other Restaurant]",E1B0-high,Kelly Driessen,100372182851002110
/ / 0573a9dd-3d07-4789-830d-e5e56fa72f97,100370666151002110,3133,3133,null,null,0573a9dd-3d07-4789-830d-e5e56fa72f97,Nicon BV,null,3133EJ,Vlaardingen,null,null,unknown,null,Unknown,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NL~EMAKINA~ANON-deeb3d50-a48e-4da0-a2fc-595e98a910e1,Nicon BV,3133EJ,3e Industriestraat 19D,Vlaardingen,OHUB,Other,Unknown,true,5,€256.0 - €282.0 (average value = 269.0),0,null,null,null,null,null,null,null,null,,null,false,0,null,null,null,unknown,null,[Other],E0B0-low,Marc Bontekoe,100370666151002110
/ / 06f1c75c-9c38-4f7b-89a0-103ee342696f,null,1681,1681,null,null,06f1c75c-9c38-4f7b-89a0-103ee342696f,STG.OMRING TBV. VZH BERKENHOF,null,1681PJ,Zwaagdyk,null,null,null,null,Unknown,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NL~FUZZIT~07FB3EFD-268E-4A62-88CD-FC6CB356B675,STG.OMRING TBV. VZH BERKENHOF,1681PJ,BOLLENMARKT 2,Zwaagdyk,NoBusinessTypeSource,Other,Unknown,true,2,€319.0 - €342.0 (average value = 327.0),0,null,null,null,null,null,null,null,null,,null,false,0,null,null,null,null,null,[Other],E0B0-high,Michael van Mierlo,null
/ / 0abb967d-8a3e-4186-aa0a-b756a9bb7d9b,null,2280,2280,null,null,0abb967d-8a3e-4186-aa0a-b756a9bb7d9b,ILSE BLOMMAERT,null,2280,Grobbendonk,null,null,null,null,Unknown,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,NL~FUZZIT~4A450346-6A4